In [138]:
import os
import pandas as pd
import numpy as np
import time 
import pwd
import re
import nltk
ximport plotly.express as px
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import sys


stop_words = set(stopwords.words("english"))
stop_words_time = ['date', 'time', 'year']
np.set_printoptions(threshold=sys.maxsize)



In [139]:
file_name = "data_urban.csv"
file_path = "../Data/" + file_name
if file_name.endswith('.csv'):
    data = pd.read_csv(file_path)
st = os.stat(file_path)

In [140]:
print(data.head())

              Sensor Name        Variable   Units            Timestamp  \
0  PER_AIRMON_MESH1907150  Particle Count  Kgm -3  2019-10-13 22:07:00   
1  PER_AIRMON_MESH1965150              O3     ppb  2019-10-13 22:07:00   
2  PER_AIRMON_MESH1965150             NO2  ugm -3  2019-10-13 22:07:00   
3  PER_AIRMON_MESH1965150              NO  ugm -3  2019-10-13 22:07:00   
4  PER_AIRMON_MESH1965150              CO  ugm -3  2019-10-13 22:07:00   

       Value  Flagged as Suspect Reading  
0    0.39000                       False  
1   16.84500                       False  
2   20.19872                       False  
3   11.43500                       False  
4  461.33424                       False  


# Preprocess data

### Remove punctuation

In [3]:
import string
def remove_punctuation(text):
    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    return text

my_string = "ana are, mere"
remove_punctuation(my_string)

'ana are  mere'

### Tokenize the data

In [147]:
from scipy import stats
from scipy.stats import shapiro

#Filter not date/is string
def filter_columns(req_string): 
    pattern  = re.compile(r'' + "|".join(stop_words_time), re.IGNORECASE)
    is_string_column = lambda x : data[x].dtypes == 'O'
    return not bool(pattern.search(req_string)) and is_string_column(req_string)

df_nlp = pd.DataFrame()
keywords = list()
for column in filter(filter_columns, data.columns):
    #Tokenize and lower case
    df_nlp[column] = data[column].apply(lambda x : x.lower().split(" "))
    vectorizer = TfidfVectorizer(stop_words="english", lowercase = False, ngram_range = (1,2))
    tfidf_matrix = vectorizer.fit_transform(df_nlp[column].astype(str).tolist())
    feature_names = vectorizer.get_feature_names()
    print(column)
    #Get the words that appear
    feature_indexes = tfidf_matrix[:,:].nonzero()[1]
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vectorizer.get_feature_names())
    df = df.apply(lambda x : x.mean()).sort_values(ascending=False)
    stat, p = shapiro(df)
    alpha = 0.05
    if p <= alpha:
        keywords.append((df[df > df.describe()['max'] - df.describe()['std']].index.tolist()))
        
df_keywords = pd.DataFrame(columns = ['keywords'])
df_keywords.loc[0, 'keywords'] = "\n".join(map(str, keywords)).astype('str') 
print(df_keywords.head())

Sensor Name
Variable
Units


AttributeError: 'str' object has no attribute 'astype'